In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.utils import to_categorical
from sklearn import preprocessing

train = pd.read_hdf("train.h5", "train")
test = pd.read_hdf("test.h5", "test")

Train=train.as_matrix()
y_train=Train[:,0]
x_train=Train[:,1:]
y_train = to_categorical(y_train,5)
Test=test.as_matrix()

# To stop potential randomness
seed = 128
rng = np.random.RandomState(seed)

#define a validation set
split_size = int(x_train.shape[0]*0.9)

train_x, val_x = x_train[:split_size], x_train[split_size:]
train_y, val_y = y_train[:split_size], y_train[split_size:]
scaler = preprocessing.StandardScaler()
scaler.fit(train_x)
train_x=scaler.transform(train_x)
val_x=scaler.transform(val_x)

In [5]:
def batch_creator(batch_size, dataset_length, dataset_name):
    """Create batch with random samples and return appropriate format"""
    batch_mask = rng.choice(dataset_length, batch_size)
    
    batch_x = eval(dataset_name + '_x')[[batch_mask]].reshape(-1, input_num_units)
    
    if dataset_name == 'train':
        batch_y = eval(dataset_name + '_y')[[batch_mask]]
        #batch_y = to_categorical(batch_y, num_classes=5)
        
    return batch_x, batch_y

In [88]:
input_num_units = 100 #nb of dimensions of the data
hidden_num_units = 500
hidden_num_units2 = 500
output_num_units = 5 #nb of classes


## defining the TF graph

#define placeholders
x = tf.placeholder(tf.float32, [None, input_num_units])
y = tf.placeholder(tf.float32, [None, output_num_units])
keep_rate = tf.placeholder("float")
# set remaining variables
training_epochs = 50
batch_size = 128
lr = 0.01
#mom = 0.9

### define weights and biases of the neural network (refer this article if you don't understand the terminologies)

weights = {
    'h1': tf.Variable(tf.random_normal([input_num_units, hidden_num_units])),
    'h2': tf.Variable(tf.random_normal([hidden_num_units, hidden_num_units2])),
#         'h3': tf.Variable(tf.random_normal([hidden_num_units2, hidden_num_units3], seed=seed)),
    'out': tf.Variable(tf.random_normal([hidden_num_units2, output_num_units]))
}

biases = {
    'h1': tf.Variable(tf.random_normal([hidden_num_units])),
    'h2': tf.Variable(tf.random_normal([hidden_num_units2])),
#         'h3': tf.Variable(tf.random_normal([hidden_num_units3], seed=seed)),
    'out': tf.Variable(tf.random_normal([output_num_units]))
}


#first hidden layer
layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['h1'])
layer_1 = tf.nn.relu(layer_1)
layer_1 = tf.nn.dropout(layer_1, keep_rate)
#second hidden layer
layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['h2'])
layer_2 = tf.nn.relu(layer_2)
layer_2 = tf.nn.dropout(layer_2, keep_rate)
#output
logits = tf.add(tf.matmul(layer_2, weights['out']), biases['out'])
#logits = tf.nn.softmax(logits)

#loss definition
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=logits))
#define optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=lr).minimize(loss)


In [89]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(training_epochs):
        avg_loss = 0.0
        total_batch = int(len(x_train) / batch_size)
        for i in range(total_batch):
            batch_x, batch_y = batch_creator(batch_size, train_x.shape[0], 'train')
            _, c = sess.run([optimizer, loss], feed_dict={x: batch_x, y: batch_y, keep_rate: 0.85})
            avg_loss += c / total_batch
        print("Epoch:", '%04d' % (epoch+1), "Loss=", \
                "{:.5f}".format(avg_loss))
    print("Optimization Finished!")
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({x: val_x, y: val_y, keep_rate: 1.0}))
    res=sess.run(logits,{x: test, keep_rate: 1.0})


Epoch: 0001 Loss= 270.62195
Epoch: 0002 Loss= 43.35669
Epoch: 0003 Loss= 14.28552
Epoch: 0004 Loss= 5.63233
Epoch: 0005 Loss= 2.51713
Epoch: 0006 Loss= 1.49170
Epoch: 0007 Loss= 1.17046
Epoch: 0008 Loss= 1.02728
Epoch: 0009 Loss= 0.92366
Epoch: 0010 Loss= 0.87705
Epoch: 0011 Loss= 0.81111
Epoch: 0012 Loss= 0.77310
Epoch: 0013 Loss= 0.77078
Epoch: 0014 Loss= 0.73772
Epoch: 0015 Loss= 0.71911
Epoch: 0016 Loss= 0.71576
Epoch: 0017 Loss= 0.70163
Epoch: 0018 Loss= 0.65233
Epoch: 0019 Loss= 0.67428
Epoch: 0020 Loss= 0.65845
Epoch: 0021 Loss= 0.64123
Epoch: 0022 Loss= 0.62754
Epoch: 0023 Loss= 0.63123
Epoch: 0024 Loss= 0.61925
Epoch: 0025 Loss= 0.63519
Epoch: 0026 Loss= 0.63372
Epoch: 0027 Loss= 0.66489
Epoch: 0028 Loss= 0.65684
Epoch: 0029 Loss= 0.64333
Epoch: 0030 Loss= 0.63548
Epoch: 0031 Loss= 0.64198
Epoch: 0032 Loss= 0.60928
Epoch: 0033 Loss= 0.63739
Epoch: 0034 Loss= 0.64969
Epoch: 0035 Loss= 0.65983
Epoch: 0036 Loss= 0.63665
Epoch: 0037 Loss= 0.61310
Epoch: 0038 Loss= 0.61710
Epoch: 0

In [26]:
#Extracting the results
tmp = np.argmax(res,axis=1)
dff = pd.DataFrame(tmp)
dff.reset_index(inplace=True)
dff.columns = ['Id','y']
dff['Id']+=45324
# dff.columns = ['Id','y']
#dff.to_csv("outputtask3.csv", header=True, index=False)

In [103]:
#%% KERAS
import numpy as np
import pandas as pd 
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Conv1D
from keras.layers.normalization import BatchNormalization

from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.utils import class_weight
#%%

DropoutRate = 0.5

model = Sequential([
    Dense(500, input_shape=(100,)),
    BatchNormalization(),
    Activation('relu'),
    Dropout(DropoutRate),
    Dense(100),
    BatchNormalization(),
    Activation('relu'),
    Dropout(DropoutRate),
    Dense(100),
    BatchNormalization(),
    Activation('relu'),
    Dropout(DropoutRate),
    
#     Dense(10),
#     Activation('relu'),
#     Dropout(DropoutRate),

#     Dense(100),
#     Activation('relu'),
#     Dropout(DropoutRate),

#     Dense(100),
#     Activation('relu'),
#     Dropout(DropoutRate),

#     Dense(100),
#     Activation('relu'),
#     Dropout(DropoutRate),

    Dense(5),
    Activation('softmax')
])



#%%
# Compile the model
import keras.backend as K

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
#%%
# Read in the data
train = pd.read_hdf("train.h5", "train")
test  = pd.read_hdf("test.h5", "test")

TrainData = train.as_matrix()
print(train.shape,TrainData.shape)

y_train=TrainData[:,0]
x_train=TrainData[:,1:]
scaler = preprocessing.StandardScaler()
scaler.fit(x_train)
x_train=scaler.transform(x_train)
#%%
# Training
from keras.utils import to_categorical
y_binary = to_categorical(y_train)

class_weight = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
model.fit(x_train, y_binary, epochs=250, batch_size=128, validation_split = 0.1, verbose=1,class_weight =class_weight )

(45324, 101) (45324, 101)
Train on 40791 samples, validate on 4533 samples
Epoch 1/250
12800/40791 [========>.....................] - ETA: 11s - loss: 1.2495 - acc: 0.5097

KeyboardInterrupt: 

In [97]:
tmp=test.as_matrix()
tmp=scaler.transform(tmp)
print(tmp.shape)
res = model.predict_classes(tmp,batch_size=1)
print(res.shape)

(8137, 100)
(8137,)


In [98]:
print(res)

[2 3 3 ... 3 1 2]


In [99]:
dff = pd.DataFrame(res)
dff.reset_index(inplace=True)
dff.columns = ['Id','y']
dff['Id']+=45324
# dff.columns = ['Id','y']
dff.to_csv("outputtask3.csv", header=True, index=False)